## Homework 2

Due: 2022-10-16 at 8:30 AM PT

# Name: Nolan Yager

For this assignment, you will practice downloadings, cleaning, and analyzing data from the [National Risk Index (NRI)](https://hazards.fema.gov/nri/) and the [CDC Social Vulnerability Index (SVI)](https://www.atsdr.cdc.gov/placeandhealth/svi/index.html).

# Preparation

1. Create a 'data' folder in the root directory of this repository.
1. Inside the 'data' folder, create a 'raw' folder.
1. Add and commit a '.gitignore' file to the root directory of this repository that excludes all contents of the 'data' folder.
1. Download the county-level NRI and SVI data for the entire United States. Place the data in the 'data/raw' folder.
1. In the repository README, provide a brief (1-2 sentence) description of each file in the 'data' folder and a link to the original source of the data.

Task 1 - NRI Data Cleaning

1. Import the NRI data. Ensure that the [FIPS code](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code) variable ('STCOFIPS') is correctly identified as a string / character variable. Otherwise, the leading zeros will be removed.


In [100]:
# Import Packages Test: 
import pandas as pd
import numpy as np

#Current WD:
import os
cwd = os.getcwd()
print(cwd)




c:\Users\nyager\Desktop\YagerMLCode\MachineLearning_Fall24\homework


In [101]:
# Loading NRI data and specifying STCOFIPS as a string column
nri_data = pd.read_csv("data/raw/NRI_Data_By_County.csv", dtype={'STCOFIPS': str})

# Here, we check if the State-County FIPS code (STCOFIPS) was loaded correctly
print(nri_data.dtypes)  # Verify if 'STCOFIPS' is treated as an object (string)
second_value = nri_data['STCOFIPS'].iloc[1]
if isinstance(second_value, str):
    print("yes Nolan, it's a string")
else:
    print("no Nolan, it isn't a string")
    


OID_                int64
NRI_ID             object
STATE              object
STATEABBRV         object
STATEFIPS           int64
                   ...   
WNTW_ALR_NPCTL    float64
WNTW_RISKV        float64
WNTW_RISKS        float64
WNTW_RISKR         object
NRI_VER            object
Length: 465, dtype: object
yes Nolan, it's a string



2. Subset the NRI data to include only the 5-digit state/county FIPS code and all colums ending with '\_AFREQ' and '\_RISKR'. Each of these columns represents a different hazard type.


In [102]:
# Subsetting: 
columns_to_keep = ['STCOFIPS'] + [col for col in nri_data.columns if col.endswith('_AFREQ') or col.endswith('_RISKR')]
nri_subset = nri_data[columns_to_keep]

# Looking at first 5 rows to check: 
print(nri_subset.head(5))


  STCOFIPS  AVLN_AFREQ      AVLN_RISKR  CFLD_AFREQ      CFLD_RISKR  \
0    01001         NaN  Not Applicable         NaN  Not Applicable   
1    01003         NaN  Not Applicable    3.684142  Relatively Low   
2    01005         NaN  Not Applicable         NaN  Not Applicable   
3    01007         NaN  Not Applicable         NaN  Not Applicable   
4    01009         NaN  Not Applicable         NaN  Not Applicable   

   CWAV_AFREQ CWAV_RISKR  DRGT_AFREQ           DRGT_RISKR  ERQK_AFREQ  ...  \
0         0.0  No Rating   25.969774       Relatively Low    0.000431  ...   
1         0.0  No Rating   12.353442  Relatively Moderate    0.000338  ...   
2         0.0  No Rating   43.956953       Relatively Low    0.000227  ...   
3         0.0  No Rating   28.894501             Very Low    0.000790  ...   
4         0.0  No Rating   28.152598       Relatively Low    0.000817  ...   

  TRND_AFREQ           TRND_RISKR TSUN_AFREQ         TSUN_RISKR VLCN_AFREQ  \
0   0.480184  Relatively Moderat


3. Create a table / dataframe that, for each hazard type, shows the number of missing values in the '\_AFREQ' and '\_RISKR' columns.


In [103]:
# Creating a dataframe:  
afreq_riskr_columns = [col for col in nri_subset.columns if col.endswith('_AFREQ') or col.endswith('_RISKR')]
missing_values = nri_subset[afreq_riskr_columns].isna().sum()

# Table: 
summary_table_missing = pd.DataFrame({
    'Hazard_Type': missing_values.index, 
    'Missing_Values': missing_values.values
})

# Print Table: 
print(summary_table_missing)

   Hazard_Type  Missing_Values
0   AVLN_AFREQ            3023
1   AVLN_RISKR               0
2   CFLD_AFREQ            2646
3   CFLD_RISKR               0
4   CWAV_AFREQ               0
5   CWAV_RISKR               0
6   DRGT_AFREQ               7
7   DRGT_RISKR               0
8   ERQK_AFREQ               0
9   ERQK_RISKR               0
10  HAIL_AFREQ               7
11  HAIL_RISKR               0
12  HWAV_AFREQ               0
13  HWAV_RISKR               0
14  HRCN_AFREQ             918
15  HRCN_RISKR               0
16  ISTM_AFREQ             229
17  ISTM_RISKR               0
18  LNDS_AFREQ              40
19  LNDS_RISKR               0
20  LTNG_AFREQ             123
21  LTNG_RISKR               0
22  RFLD_AFREQ               0
23  RFLD_RISKR               0
24  SWND_AFREQ               7
25  SWND_RISKR               0
26  TRND_AFREQ               7
27  TRND_RISKR               0
28  TSUN_AFREQ            3103
29  TSUN_RISKR               0
30  VLCN_AFREQ            3125
31  VLCN

4. Create a new column in the original data table indicating whether or not 'AVLN_AFREQ' is missing or observed. Show the cross-tabulation of the 'AVLN_AFREQ' missingness and 'AVLN_RISKR' columns (including missing values). What do you observe?

In [105]:
# Create a new column 
nri_data['AVLN_AFREQ_Missing'] = nri_data['AVLN_AFREQ'].isna()

# Cross-tab set up
cross_tab = pd.crosstab(nri_data['AVLN_AFREQ_Missing'], nri_data['AVLN_RISKR'], dropna=False)

# Printing
print(cross_tab)

AVLN_RISKR          Not Applicable  Relatively High  Relatively Low  \
AVLN_AFREQ_Missing                                                    
False                            0               15              52   
True                          3023                0               0   

AVLN_RISKR          Relatively Moderate  Very High  Very Low  
AVLN_AFREQ_Missing                                            
False                                33          9        99  
True                                  0          0         0  


5. Assuming that a risk that is "not applicable" to a county has an annualized frequency of 0, impute the relevant missing values in the '\_AFREQ' columns with 0.

In [90]:
# code here

Task 2 - SVI Data Cleaning

1. Import the SVI data. Ensure that the FIPS code is correctly identified as a string / character variable. Otherwise, the leading zeros will be removed.
1. Subset the SVI data to include only the following columns:
`ST, STATE, ST_ABBR, STCNTY, COUNTY, FIPS, LOCATION, AREA_SQMI, E_TOTPOP, EP_POV150, EP_UNEMP, EP_HBURD, EP_NOHSDP, EP_UNINSUR, EP_AGE65, EP_AGE17, EP_DISABL, EP_SNGPNT, EP_LIMENG, EP_MINRTY, EP_MUNIT, EP_MOBILE, EP_CROWD, EP_NOVEH, EP_GROUPQ, EP_NOINT, EP_AFAM, EP_HISP, EP_ASIAN, EP_AIAN, EP_NHPI, EP_TWOMORE, EP_OTHERRACE`

In [91]:
# code here

2. Create a table / dataframe that shows the number of missing values in each column.
(Hint: if you wrote a function for Task 1, you can reuse it here.)

In [92]:
#code

Task 3 - Data Merging
1. Identify any FIPS codes that are present in the NRI data but not in the SVI data and vice versa. Describe any discrepancies and possible causes? What to these discrepancies, if any, mean for interpreting results based on the merged dataset moving forward?

In [93]:
#code

2. Merge the NRI and SVI data on the FIPS code. Use an outer join to keep all counties in the final dataset.

In [94]:
#code

3. Create a table / dataframe that shows the number of missing values in each column of the merged dataset.

In [95]:
#code

Task 4 - Data Analysis

1. For each numerical variable in the merged dataset, plot a histogram showing the distribution of values.
(Hint: write a function to make the histogram for a single variable, then use a loop or apply function to make the histograms for all numerical variables.)

In [96]:
#code

END OF HOMEWORK 2. 